In [13]:
using StochasticPrograms
using GLPK

In [14]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, x₁ >= 40)
        @decision(simple_model, x₂ >= 20)
        @objective(simple_model, Min, 100*x₁ + 150*x₂)
        @constraint(simple_model, x₁ + x₂ <= 120)
    end
    @stage 2 begin
        @uncertain q₁ q₂ d₁ d₂
        @recourse(simple_model, 0 <= y₁ <= d₁)
        @recourse(simple_model, 0 <= y₂ <= d₂)
        @objective(simple_model, Max, q₁*y₁ + q₂*y₂)
        @constraint(simple_model, 6*y₁ + 10*y₂ <= 60*x₁)
        @constraint(simple_model, 8*y₁ + 5*y₂ <= 80*x₂)
    end
end


Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [15]:
ξ₁ = @scenario q₁ = 24.0 q₂ = 28.0 d₁ = 500.0 d₂ = 100.0 probability = 0.4
ξ₂ = @scenario q₁ = 28.0 q₂ = 32.0 d₁ = 300.0 d₂ = 300.0 probability = 0.6

Scenario with probability 0.6
  q₁: 28.0
  q₂: 32.0
  d₁: 300.0
  d₂: 300.0

In [16]:
sp = instantiate(simple_model, [ξ₁, ξ₂], optimizer = GLPK.Optimizer)

Stochastic program with:
 * 2 decision variables
 * 2 recourse variables
 * 2 scenarios of type Scenario
Structure: Deterministic equivalent
Solver name: GLPK

In [17]:
print(sp)

Deterministic equivalent problem
Min 100 x₁ + 150 x₂ - 9.600000000000001 y₁₁ - 11.200000000000001 y₂₁ - 16.8 y₁₂ - 19.2 y₂₂
Subject to
 x₁ in Decisions
 x₂ in Decisions
 y₁₁ in RecourseDecisions
 y₂₁ in RecourseDecisions
 y₁₂ in RecourseDecisions
 y₂₂ in RecourseDecisions
 x₁ >= 40.0
 x₂ >= 20.0
 y₁₁ >= 0.0
 y₂₁ >= 0.0
 y₁₂ >= 0.0
 y₂₂ >= 0.0
 x₁ + x₂ <= 120.0
 -60 x₁ + 6 y₁₁ + 10 y₂₁ <= 0.0
 -80 x₂ + 8 y₁₁ + 5 y₂₁ <= 0.0
 -60 x₁ + 6 y₁₂ + 10 y₂₂ <= 0.0
 -80 x₂ + 8 y₁₂ + 5 y₂₂ <= 0.0
 y₁₁ <= 500.0
 y₂₁ <= 100.0
 y₁₂ <= 300.0
 y₂₂ <= 300.0
Solver name: GLPK

In [6]:
optimize!(sp)

In [7]:
sp_lshaped = instantiate(simple_model, [ξ₁, ξ₂], optimizer = LShaped.Optimizer)

Stochastic program with:
 * 2 decision variables
 * 2 recourse variables
 * 2 scenarios of type Scenario
Structure: Stage-decomposition
Solver name: L-shaped with disaggregate cuts

In [8]:
print(sp_lshaped)

First-stage 
Min 100 x₁ + 150 x₂
Subject to
 x₁ in Decisions
 x₂ in Decisions
 x₁ >= 40.0
 x₂ >= 20.0
 x₁ + x₂ <= 120.0

Second-stage 
Subproblem 1 (p = 0.40):
Max 24 y₁ + 28 y₂
Subject to
 x₁ in Known(value = 40.0)
 x₂ in Known(value = 20.0)
 y₁ in RecourseDecisions
 y₂ in RecourseDecisions
 y₁ >= 0.0
 y₂ >= 0.0
 y₁ <= 500.0
 y₂ <= 100.0
 -60 x₁ + 6 y₁ + 10 y₂ <= 0.0
 -80 x₂ + 8 y₁ + 5 y₂ <= 0.0

Subproblem 2 (p = 0.60):
Max 28 y₁ + 32 y₂
Subject to
 x₁ in Known(value = 40.0)
 x₂ in Known(value = 20.0)
 y₁ in RecourseDecisions
 y₂ in RecourseDecisions
 y₁ >= 0.0
 y₂ >= 0.0
 y₁ <= 300.0
 y₂ <= 300.0
 -60 x₁ + 6 y₁ + 10 y₂ <= 0.0
 -80 x₂ + 8 y₁ + 5 y₂ <= 0.0

Solver name: L-shaped with disaggregate cuts

In [9]:
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), GLPK.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), GLPK.Optimizer)

In [10]:
optimize!(sp_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\eangelino\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
L-Shaped Gap  Time: 0:00:01 (6 iterations)
  Objective:       -855.8333333333358
  Gap:             0.0
  Number of cuts:  8
  Iterations:      6


In [11]:
print(sp_lshaped)

First-stage 
Min 100 x₁ + 150 x₂ + θ₁ + θ₂
Subject to
 x₁ in Decisions
 x₂ in Decisions
 x₁ >= 40.0
 x₂ >= 20.0
 x₁ + x₂ <= 120.0
 179.2 x₂ + θ₁ >= 0.0
 307.2 x₂ + θ₂ >= 0.0
 96 x₁ + θ₁ >= 0.0
 115.19999999999999 x₁ + θ₂ >= -1584.0
 96.00000000000001 x₂ + θ₁ >= -520.0
 168 x₂ + θ₂ >= -2610.000000000001
 83.51999999999998 x₁ + 84.48000000000005 x₂ + θ₂ >= 0.0
 49.92 x₁ + 46.08 x₂ + θ₁ >= 0.0

Second-stage 
Subproblem 1 (p = 0.40):
Max 24 y₁ + 28 y₂
Subject to
 x₁ in Known(value = 46.666666666666664)
 x₂ in Known(value = 36.24999999999999)
 y₁ in RecourseDecisions
 y₂ in RecourseDecisions
 y₁ >= 0.0
 y₂ >= 0.0
 y₁ <= 500.0
 y₂ <= 100.0
 -60 x₁ + 6 y₁ + 10 y₂ <= 0.0
 -80 x₂ + 8 y₁ + 5 y₂ <= 0.0

Subproblem 2 (p = 0.60):
Max 28 y₁ + 32 y₂
Subject to
 x₁ in Known(value = 46.666666666666664)
 x₂ in Known(value = 36.24999999999999)
 y₁ in RecourseDecisions
 y₂ in RecourseDecisions
 y₁ >= 0.0
 y₂ >= 0.0
 y₁ <= 300.0
 y₂ <= 300.0
 -60 x₁ + 6 y₁ + 10 y₂ <= 0.0
 -80 x₂ + 8 y₁ + 5 y₂ <= 0.0

Solve

In [12]:
optimal_decision(sp_lshaped)

2-element Vector{Float64}:
 46.666666666666664
 36.24999999999999